In [1]:
from torchvision import datasets
from matplotlib import pyplot as plt
import torch
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np

data_path = 'C:/Users/keith/Desktop/School/RTAI/HW/HW4'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True) # <1>
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True) # <2>

In [2]:
transformed_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                      transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                               (0.2470, 0.2435, 0.2616))
                                      ]))

transformed_cifar10_val = datasets.CIFAR10(data_path, train=False, download=False,
                                      transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                               (0.2470, 0.2435, 0.2616))
                                      ]))

In [3]:
label_map = {6: 0, 7: 1, 8: 2, 9: 3}
class_names = ['frog','horse','ship','truck']
cifar4 = [(img, label_map[label])
          for img, label in transformed_cifar10 
          if label in [6, 9]]
cifar4_val = [(img, label_map[label])
              for img, label in transformed_cifar10_val
              if label in [6, 9]]

In [4]:
model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 4),
            nn.LogSoftmax(dim=1))

learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()

n_epochs = 200

train_loader = torch.utils.data.DataLoader(cifar4, batch_size=64, shuffle=True)

import time

start = time.time()
for epoch in range(n_epochs):
    for img, label in train_loader:
        
        batch_size = img.shape[0]
        out = model(img.view(batch_size, -1))
        loss = loss_fn(out, label)
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 20 == 0 or epoch == 199:
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
end = time.time()

ttime = end - start
print(str(ttime) + " seconds")

Epoch: 0, Loss: 0.842224
Epoch: 20, Loss: 0.012303
Epoch: 40, Loss: 0.002619
Epoch: 60, Loss: 0.000972
Epoch: 80, Loss: 0.000866
Epoch: 100, Loss: 0.000859
Epoch: 120, Loss: 0.000103
Epoch: 140, Loss: 0.000490
Epoch: 160, Loss: 0.000057
Epoch: 180, Loss: 0.000280
Epoch: 199, Loss: 0.000062
626.0050208568573 seconds


In [5]:
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 1.000000


In [6]:
val_loader = torch.utils.data.DataLoader(cifar4_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.911500


In [7]:
numel_list = [p.numel()
              for p in model.parameters()
              if p.requires_grad == True]
sum(numel_list), numel_list

(3737732, [3145728, 1024, 524288, 512, 65536, 128, 512, 4])